# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,21.97,86,100,2.24,EC,1731456148
1,1,vorkuta,67.5000,64.0000,-25.31,100,100,1.68,RU,1731456149
2,2,sitka,57.0531,-135.3300,5.72,93,100,1.49,US,1731456150
3,3,adamstown,-25.0660,-130.1015,21.55,73,100,4.83,PN,1731456151
4,4,bilibino,68.0546,166.4372,-25.38,99,24,1.28,RU,1731456153


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot

plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,tshabong,-26.0500,22.4500,23.50,13,0,3.06,BW,1731456224
115,115,ad dilam,23.9915,47.1627,22.60,27,0,2.19,SA,1731456299
173,173,remire-montjoly,4.9167,-52.2667,26.02,94,0,1.03,GF,1731456368
238,238,adrar,20.5022,-10.0711,22.77,16,0,2.33,MR,1731456447
279,279,mangrol,21.1167,70.1167,25.46,53,0,4.02,IN,1731456498


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,86,
1,vorkuta,RU,67.5000,64.0000,100,
2,sitka,US,57.0531,-135.3300,93,
3,adamstown,PN,-25.0660,-130.1015,73,
4,bilibino,RU,68.0546,166.4372,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000  # Set search radius to 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",  # Search category for hotels
    "limit": 1,  # Limit results to the first hotel found
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
vorkuta - nearest hotel: "Мегаполис"
sitka - nearest hotel: Aspen Suites Hotel
adamstown - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Pousada Mediterrâneo
nantucket - nearest hotel: Faraway Nantucket
arraial do cabo - nearest hotel: No hotel found
shache - nearest hotel: 莎车新盛宾馆
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
waitangi - nearest hotel: Hotel Chathams
mahibadhoo - nearest hotel: No hotel found
jayapura - nearest hotel: Hotel Jasmine Jayapura
greymouth - nearest hotel: Recreation Hotel
bredasdorp - nearest hotel: Victoria Hotel
rongelap - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
port mathurin - nearest hotel: Escale Vacances
port-aux-francais - nearest hotel: Keravel
bodo - nearest hotel: Zefyr hotell
mitu - nearest hotel: Hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,86,Hostal La Mirada De Solitario George
1,vorkuta,RU,67.5000,64.0000,100,"""Мегаполис"""
2,sitka,US,57.0531,-135.3300,93,Aspen Suites Hotel
3,adamstown,PN,-25.0660,-130.1015,73,No hotel found
4,bilibino,RU,68.0546,166.4372,99,No hotel found
...,...,...,...,...,...,...
543,tanjung pandan,ID,-2.7500,107.6500,80,Billiton Hotel & Club
544,keflavik,IS,64.0049,-22.5624,70,Núpan Deluxe
545,santa marta,CO,11.2408,-74.1990,77,Casa Mita
546,terrace,CA,54.5163,-128.6035,100,Sandman Hotel Terrace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
plot_1 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)